In [ ]:
!pip list

In [1]:
import evalml
from evalml import AutoMLSearch
from evalml.objectives import FraudCost

In [2]:
fraud_objective = FraudCost(
    retry_percentage=0.5,
    interchange_fee=0.02,
    fraud_payout_percentage=0.75,
    amount_col="amount",
)

In [3]:
X, y = evalml.demos.load_fraud(n_rows=5000)

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 5000
Targets
False    86.20%
True     13.80%
Name: count, dtype: object


d:\github\8-forked-repos\evalml\.venv\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
d:\github\8-forked-repos\evalml\.venv\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
d:\github\8-forked-repos\evalml\.venv\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
d:\github\8-forked-repos\evalml\.venv\Lib\site-packages\woodwork\type_sys\utils.py:33: UserWarning: Could not infer format, so each e

In [4]:
cols_to_drop = ["datetime", "expiration_date", "country", "region", "provider"]
for col in cols_to_drop:
    X.ww.pop(col)

X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(
    X, y, problem_type="binary", test_size=0.2, random_seed=0
)

X.ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
card_id,int64,Integer,['numeric']
store_id,int64,Integer,['numeric']
amount,int64,Integer,['numeric']
currency,category,Categorical,['category']
customer_present,bool,Boolean,[]
lat,float64,Double,['numeric']
lng,float64,Double,['numeric']


In [5]:
automl = AutoMLSearch(
    X_train=X_train,
    y_train=y_train,
    problem_type="binary",
    objective=fraud_objective,
    additional_objectives=["auc", "f1", "precision"],
    allowed_model_families=["random_forest", "linear_model"],
    max_batches=1,
    optimize_thresholds=True,
    verbose=True,
)

automl.search(interactive_plot=False)

AutoMLSearch will use mean CV score to rank pipelines.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Cost. 
Lower score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of None pipelines. 
Allowed model families: 

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.790

*****************************
* Evaluating Batch Number 1 *
*****************************

Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model:
	Starting cross validation
	Finished cross validation - mean Fraud Cost: 0.008

Search finished after 5.20 seconds     
Best pipeline: Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model


{1: {'Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model': 4.021155595779419,
  'Total time of batch': 4.148508310317993}}

In [6]:
automl.rankings

,id,pipeline_name,search_order,ranking_score,mean_cv_score,standard_deviation_cv_score,percent_better_than_baseline,high_variance_cv,parameters
0,1,Random Forest Classifier w/ Label Encoder + Im...,1,0.008125,0.008125,0.000636,78.152346,False,"{'Label Encoder': {'positive_label': None}, 'I..."
1,0,Mode Baseline Binary Classification Pipeline,0,0.789648,0.789648,0.001136,0.000000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


In [7]:
automl.describe_pipeline(automl.rankings.iloc[1]["id"])


************************************************
* Mode Baseline Binary Classification Pipeline *
************************************************

Problem Type: binary
Model Family: Baseline

Pipeline Steps
1. Label Encoder
	 * positive_label : None
2. Baseline Classifier
	 * strategy : mode

Training
Training for binary problems.
Objective to optimize binary classification pipeline thresholds for: <evalml.objectives.fraud_cost.FraudCost object at 0x000001D1D7063860>
Total training time (including CV): 0.7 seconds

Cross Validation
----------------
             Fraud Cost   AUC    F1  Precision # Training # Validation
0                 0.791 0.500 0.000      0.000      2,666        1,334
1                 0.789 0.500 0.000      0.000      2,667        1,333
2                 0.789 0.500 0.000      0.000      2,667        1,333
mean              0.790 0.500 0.000      0.000          -            -
std               0.001 0.000 0.000      0.000          -            -
coef of var      

In [9]:
best_pipeline = automl.best_pipeline

In [10]:
best_pipeline.score(X_holdout, y_holdout, objectives=["auc", fraud_objective])

OrderedDict([('AUC', 0.8750462355829045), ('Fraud Cost', 0.00827478082087425)])

# Why optimize for a problem-specific objective?

In [11]:
automl_auc = AutoMLSearch(
    X_train=X_train,
    y_train=y_train,
    problem_type="binary",
    objective="auc",
    additional_objectives=["f1", "precision"],
    max_batches=1,
    allowed_model_families=["random_forest", "linear_model"],
    optimize_thresholds=True,
    verbose=True,
)

automl_auc.search(interactive_plot=False)

AutoMLSearch will use mean CV score to rank pipelines.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of None pipelines. 
Allowed model families: 

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean AUC: 0.500

*****************************
* Evaluating Batch Number 1 *
*****************************

Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model:
	Starting cross validation
	Finished cross validation - mean AUC: 0.845

Search finished after 4.33 seconds     
Best pipeline: Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model
Best pipeline AUC: 

{1: {'Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + RF Classifier Select From Model': 3.748507022857666,
  'Total time of batch': 3.893397569656372}}

In [12]:
automl_auc.rankings

,id,pipeline_name,search_order,ranking_score,mean_cv_score,standard_deviation_cv_score,percent_better_than_baseline,high_variance_cv,parameters
0,1,Random Forest Classifier w/ Label Encoder + Im...,1,0.844675,0.844675,0.006389,34.467534,False,"{'Label Encoder': {'positive_label': None}, 'I..."
1,0,Mode Baseline Binary Classification Pipeline,0,0.500000,0.500000,0.000000,0.000000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


In [13]:
best_pipeline_auc = automl_auc.best_pipeline

In [14]:
# get the fraud score on holdout data
best_pipeline_auc.score(X_holdout, y_holdout, objectives=["auc", fraud_objective])

OrderedDict([('AUC', 0.8750462355829045), ('Fraud Cost', 0.02574801808280861)])

In [15]:
# fraud score on fraud optimized again
best_pipeline.score(X_holdout, y_holdout, objectives=["auc", fraud_objective])

OrderedDict([('AUC', 0.8750462355829045), ('Fraud Cost', 0.00827478082087425)])